# Trace collection of SCA-Protected Implementation of Ascon128

## Setup Target Board

In [1]:
PLATFORM = 'CW308_STM32F3'
CRYPTO_TARGET = 'NONE'
SS_VER = 'SS_VER_1_1'

BAUD_RATE = 38400

## Detect Chipwhisperer Board

In [ ]:
import time
import chipwhisperer as cw

scope = cw.scope()

target_type = cw.targets.SimpleSerial
try:
    target = cw.target(scope, target_type)
except:
    print("INFO: Caught exception on reconnecting to target - attempting to reconnect to scope first.")
    print("INFO: This is a work-around when USB has died without Python knowing. Ignore errors above this line.")
    scope = cw.scope()
    target = cw.target(scope, target_type)

print("INFO: Found ChipWhisperer😍")

prog = cw.programmers.STM32FProgrammer

time.sleep(0.05)
scope.default_setup()
def reset_target(scope):
    scope.io.nrst = 'low'
    time.sleep(0.05)
    scope.io.nrst = 'high_z'
    time.sleep(0.05)


### Setup Target Algorithm

In [2]:
CRYPTO_PATH = 'crypto_aead/ascon128v12/protected_bi32_armv6'
DATA_LEN = 190
RESP_LEN = 96
n_shares = 2
SS_SHARED = 1
NUM_SHARES_KEY = n_shares
NUM_SHARES_NPUB = n_shares
NUM_SHARES_AD = n_shares
NUM_SHARES_M = n_shares
NUM_SHARES_C = n_shares
ASCON_ROR_SHARES = 5
ASCON_EXTERN_BI = 0
ASCON_PA_ROUNDS = 12
ASCON_PB_ROUNDS = 6

In [3]:
EXTRA_OPTS = ' -DNUM_SHARES_KEY=' + str(NUM_SHARES_KEY)
EXTRA_OPTS += ' -DNUM_SHARES_NPUB=' + str(NUM_SHARES_NPUB)
EXTRA_OPTS += ' -DNUM_SHARES_AD=' + str(NUM_SHARES_AD)
EXTRA_OPTS += ' -DNUM_SHARES_M=' + str(NUM_SHARES_M)
EXTRA_OPTS += ' -DNUM_SHARES_C=' + str(NUM_SHARES_C)
EXTRA_OPTS += ' -DASCON_ROR_SHARES=' + str(ASCON_ROR_SHARES)
EXTRA_OPTS += ' -DASCON_EXTERN_BI=' + str(ASCON_EXTERN_BI)
EXTRA_OPTS += ' -DASCON_PA_ROUNDS=' + str(ASCON_PA_ROUNDS)
EXTRA_OPTS += ' -DASCON_PB_ROUNDS=' + str(ASCON_PB_ROUNDS)
EXTRA_OPTS += ' -DSS_SHARED=' + str(SS_SHARED)
EXTRA_OPTS += ' -DDATA_LEN=' + str(DATA_LEN)
EXTRA_OPTS += ' -DRESP_LEN=' + str(RESP_LEN)

## Compile

In [4]:
%%bash -s "$SS_VER" "$PLATFORM" "$CRYPTO_TARGET" "$CRYPTO_PATH" "$EXTRA_OPTS"
PATH_TO_YOUR_CHIPWHISPERER_FOLDER=`pwd`
export notebook_dir=$PWD
mkdir $PATH_TO_YOUR_CHIPWHISPERER_FOLDER/hardware/victims/firmware/simpleserial-ascon
cp -r simpleserial-ascon $PATH_TO_YOUR_CHIPWHISPERER_FOLDER/hardware/victims/firmware/
cd $PATH_TO_YOUR_CHIPWHISPERER_FOLDER/hardware/victims/firmware/simpleserial-ascon
rm -rf objdir-$2/Implementations/$4 objdir/Implementations/$4 objdir .dep
mkdir -p objdir-$2/Implementations/$4 objdir/Implementations/$4
make SS_VER=$1 PLATFORM=$2 CRYPTO_TARGET=$3 CRYPTO_PATH=$4 EXTRA_OPTS="$5"
cp simpleserial-ascon-$2.hex $notebook_dir/simpleserial-ascon-$2.hex

mkdir: cannot create directory ‘/home/asteroid/chipwhisperer/hardware/victims/firmware/simpleserial-ascon’: File exists
cp: cannot stat 'simpleserial-ascon': No such file or directory


Building for platform CW308_STM32F3 with CRYPTO_TARGET=NONE
SS_VER set to SS_VER_1_1
make clean_objs .dep 
make[1]: Entering directory '/home/asteroid/chipwhisperer/hardware/victims/firmware/simpleserial-ascon'
Building for platform CW308_STM32F3 with CRYPTO_TARGET=NONE
SS_VER set to SS_VER_1_1
rm -f -- simpleserial-ascon-CW308_STM32F3.hex
rm -f -- simpleserial-ascon-CW308_STM32F3.eep
rm -f -- simpleserial-ascon-CW308_STM32F3.cof
rm -f -- simpleserial-ascon-CW308_STM32F3.elf
rm -f -- simpleserial-ascon-CW308_STM32F3.map
rm -f -- simpleserial-ascon-CW308_STM32F3.sym
rm -f -- simpleserial-ascon-CW308_STM32F3.lss
rm -f -- objdir-CW308_STM32F3/*.o
rm -f -- objdir-CW308_STM32F3/*.lst
rm -f -- simpleserial-ascon.s randombytes.s Implementations/crypto_aead/ascon128v12/protected_bi32_armv6/aead.s Implementations/crypto_aead/ascon128v12/protected_bi32_armv6/constants.s Implementations/crypto_aead/ascon128v12/protected_bi32_armv6/crypto_aead.s Implementations/crypto_aead/ascon128v12/protected_bi

## Flash Binary on the Victim Board

**Note:** If you use a STM32F target board of Revision-02 then you first need to set a specific jumper cable on the UFO board for the following code to work.

For more information have a look here: https://rtfm.newae.com/Targets/UFO%20Targets/CW308T-STM32F/

In [ ]:
HEXFILE = "simpleserial-ascon-" + PLATFORM + ".hex"
cw.program_target(scope, prog, HEXFILE)

## Interface Constants

In [5]:
import numpy as np
import random

# Command
CMD_ASCON = 'a'
CMD_ECHO_TEST = 'e'

# Flags
M = 0x01              # data contains message 
C = 0x02              # data contains ciphertext
A = 0x04              # data contains associated data
N = 0x08              # data contains nonce
K = 0x10              # data contains key
RUN_ENC = 0x20        # run enc after data transmission
RUN_DEC = 0x40        # run dec after data transmission
OMIT_RESULT = 0x80    # omit returning result after encryption/decryption

## Interface Functions

In [7]:
import sys
sys.path.insert(0,"../helpers")
from ascon_helper import cw_generate_shares as generate_shares
from ascon_helper import cw_combine_shares as combine_shares
from ascon_helper import size_shares

## Test one encryption

In [ ]:
# Define input data
m = bytearray.fromhex('00010203')
a = bytearray.fromhex('00010203')
n = bytearray.fromhex('000102030405060708090a0b0c0d0e0f')
k = bytearray.fromhex('000102030405060708090a0b0c0d0e0f')

scope.clock.adc_src = 'clkgen_x4'
scope.gain.db = 28

data = bytearray([N|K|M|A|RUN_ENC])
data += bytearray([len(m)]) + generate_shares(m, NUM_SHARES_M)
data += bytearray([len(a)]) + generate_shares(a, NUM_SHARES_AD)
data += generate_shares(n, NUM_SHARES_NPUB)
data += generate_shares(k, NUM_SHARES_KEY)
data += bytearray.fromhex('00'*(DATA_LEN-len(data)))
target.flush()
print('d: ' + ''.join('{:02x}'.format(x) for x in data))
scope.arm()
target.simpleserial_write(CMD_ASCON,data)
scope.capture()
response = target.simpleserial_read('r', RESP_LEN, end='\n', timeout=250, ack=True)

# Extract and print response
mlen = size_shares(len(m), NUM_SHARES_C)
c = combine_shares(response[0:mlen], NUM_SHARES_C)
t = combine_shares(response[mlen:mlen + 16*NUM_SHARES_C], NUM_SHARES_C)
print('c: ' + ''.join('{:02x}'.format(x) for x in c[:len(m)]))
print('t: ' + ''.join('{:02x}'.format(x) for x in t))

scope.adc.samples = scope.adc.trig_count

## Set parameters for trace collection

In [8]:
PTlen = 4
CTlen = PTlen
TAGlen = 16
ADlen = 4
Nlen = 16
Klen = 16

# Set fixed parameters
fixed_key       = bytearray(np.random.randint(0,256,Klen, dtype=np.uint8))
fixed_nonce     = bytearray(np.random.randint(0,256,Nlen, dtype=np.uint8))
fixed_ad        = bytearray(np.random.randint(0,256,ADlen, dtype=np.uint8))
fixed_message   = bytearray(np.random.randint(0,256,PTlen, dtype=np.uint8))

In [ ]:
import trsfile
import sys
import matplotlib.pyplot as plt
from tqdm import tqdm
import os

# Path to your .trs file
file_path = "ascon_cw_protected.trs"
# Open the .trs file
with trsfile.TraceSet(file_path, mode='r') as trs:
    keys = []
    nonces = []
    messages = []
    ads = []

    # Iterate through all traces
    for trace in tqdm(trs):
        keys+=[bytearray(np.array(trace.parameters["key"].value,dtype=np.uint8))]
        nonces+=[bytearray(np.array(trace.parameters["nonce"].value,dtype=np.uint8))]
        messages+=[bytearray(np.array(trace.parameters["plaintext"].value,dtype=np.uint8))]
        ads+=[bytearray(np.array(trace.parameters["associated_data"].value,dtype=np.uint8))]



In [ ]:


COMPRESS_SAMPLES = True

#Set scope
n_traces = 1000
try:
    samples_per_trace = scope.adc.samples
except NameError:
    samples_per_trace = 5712

target.flush()
print(scope.adc.samples)

if COMPRESS_SAMPLES:
    sample_coding = trsfile.SampleCoding.SHORT
else:
    sample_coding = trsfile.SampleCoding.FLOAT

# Create tracefile
current_time = time.strftime("%Y%m%d_%H%M%S",time.localtime())
traceFileName = "traces/" + "ASCON_cw_P{}_generated".format(n_shares) + "{}_{}".format(n_traces, current_time) + ".trs"

headers = {
    trsfile.Header.TRS_VERSION:2,
    trsfile.Header.DESCRIPTION:"Ascon cw order {} masked traces".format(n_shares),
    trsfile.Header.NUMBER_SAMPLES:int(samples_per_trace),
    trsfile.Header.LENGTH_DATA:size_shares(Klen,n_shares)+size_shares(Nlen,n_shares)+size_shares(PTlen,n_shares)+size_shares(ADlen,n_shares)+size_shares(CTlen,n_shares)+size_shares(TAGlen,n_shares),
    trsfile.Header.SAMPLE_CODING:sample_coding,
    trsfile.Header.LABEL_X:"s",
    trsfile.Header.LABEL_Y:"V",
    trsfile.Header.TRACE_PARAMETER_DEFINITIONS:trsfile.parametermap.TraceParameterDefinitionMap(
            {'key': trsfile.traceparameter.TraceParameterDefinition(trsfile.traceparameter.ParameterType.BYTE, size_shares(Klen,n_shares), 0),
             'nonce': trsfile.traceparameter.TraceParameterDefinition(trsfile.traceparameter.ParameterType.BYTE, size_shares(Nlen,n_shares), size_shares(Klen,n_shares)),
             'plaintext': trsfile.traceparameter.TraceParameterDefinition(trsfile.traceparameter.ParameterType.BYTE, size_shares(PTlen,n_shares), size_shares(Klen,n_shares)+size_shares(Nlen,n_shares)),
             'associated_data': trsfile.traceparameter.TraceParameterDefinition(trsfile.traceparameter.ParameterType.BYTE, size_shares(ADlen,n_shares), size_shares(Klen,n_shares)+size_shares(Nlen,n_shares)+size_shares(PTlen,n_shares)),
             'ciphertext': trsfile.traceparameter.TraceParameterDefinition(trsfile.traceparameter.ParameterType.BYTE, size_shares(CTlen,n_shares), size_shares(Klen,n_shares)+size_shares(Nlen,n_shares)+size_shares(PTlen,n_shares)+size_shares(ADlen,n_shares)),
             'tag': trsfile.traceparameter.TraceParameterDefinition(trsfile.traceparameter.ParameterType.BYTE, size_shares(TAGlen,n_shares), size_shares(Klen,n_shares)+size_shares(Nlen,n_shares)+size_shares(PTlen,n_shares)+size_shares(ADlen,n_shares)+size_shares(CTlen,n_shares)),
             }),
}
traceFile = trsfile.trs_open(traceFileName, mode='w', headers=headers)
    
# Main loop
start_time = time.time()
for numTotal in tqdm(range(n_traces),desc="collect traces"):
    try:
        flags = M|A|N|K|RUN_ENC
        data = bytearray([flags])
        nonce = nonces[numTotal]
        message = messages[numTotal]
        ad = ads[numTotal]
        key=keys[numTotal]
        masked_plaintext = generate_shares(message, NUM_SHARES_M)
        data += bytearray([len(message)]) + masked_plaintext
        masked_ad = generate_shares(ad, NUM_SHARES_AD)
        data += bytearray([len(ad)]) + masked_ad
        masked_nonce = generate_shares(nonce, NUM_SHARES_NPUB)
        data += masked_nonce
        masked_key = generate_shares(key, NUM_SHARES_KEY)
        data += masked_key
        if len(data) > DATA_LEN:
            print(len(data))
        assert(len(data) <= DATA_LEN)
        data += bytearray.fromhex('00'*(DATA_LEN-len(data)))
        
        # write data and collect trace
        scope.arm()
        target.simpleserial_write(CMD_ASCON,data)
        scope.capture()
        trace = scope.get_last_trace(as_int=COMPRESS_SAMPLES)

        response = target.simpleserial_read('r', RESP_LEN, end='\n', timeout=250, ack=True)

        # Extract and print response
        mlen = size_shares(len(message), NUM_SHARES_C)
        masked_ct = response[0:mlen]
        masked_tag = response[mlen:mlen + 16*NUM_SHARES_C]

        if numTotal == 0:
            if samples_per_trace != scope.adc.trig_count:
                print("Number of collected samples is non optimal: \n\rCollected:{}\n\rTrigger:{}".format(samples_per_trace ,scope.adc.trig_count))
    
        ## Write data and trace in .TRS file
        traceFile.append(trsfile.Trace(sample_coding,
                                        trace,
                                        trsfile.parametermap.TraceParameterMap({'key': trsfile.parametermap.ByteArrayParameter(masked_key),
                                                                            'nonce': trsfile.parametermap.ByteArrayParameter(masked_nonce),
                                                                            'plaintext': trsfile.parametermap.ByteArrayParameter(masked_plaintext),
                                                                            'ad': trsfile.parametermap.ByteArrayParameter(masked_ad),
                                                                            'ct': trsfile.parametermap.ByteArrayParameter(masked_ct),
                                                                            'tag': trsfile.parametermap.ByteArrayParameter(masked_tag),
                                                                            })
                                ))

    except Exception as ex:
        print("ERROR: ", ex)
        sys.exit(-1)

traceFile.close()

print("Done: " + str(numTotal))
print("Total time: " + str(time.time() - start_time))

## Disconnect from Board

In [ ]:
scope.dis()
target.dis()